# Data Preprocessing Tools

## Importing the libraries

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [16]:
dataset = pd.read_csv('Data.csv') # pandas dataframe - each column is a categorical variable

# Every machine learning model has features and labels.
# Features(independent variables) are inputs, label(dependent variables/target) are output

# last column --> labels, all columns except last column --> features

# X --> features, y --> label

# iloc[r:r,c:c]

# [:] --> no range, means take everything 
# [:, :-1] --> take all rows, all columns excluding last one (uppper bound not included in slicing)
# [:, -1] --> take all rows, of last column 
# -1 --> index of last column 

# Create maxtrix of features and label variables 

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [17]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [18]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [19]:
# Checking for missing data

np.isnan(np.sum(X[:, 1:3]))

True

In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')


# select only the numberical columns 

imputer.fit(X[:, 1:3]) # fit() will look at NaN values, and will compute mean
X[:, 1:3] = imputer.transform(X[:, 1:3]) # transform() will replace NaN by mean(replace)

# update columns with returned by transform()

In [21]:
np.isnan(np.sum(X[:, 1:3]))

False

In [22]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

### Encoding the Independent Variable

In [23]:
# Ml models cannot learn from strings. It can only learn from binary numbers - 0 and 1
# so we will convert all columns of str to int 

# note: some columns have type object. Conversion --> object to str to int

# convert categorical variables to dummy variables (0 or 1)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# [0,1,2] --> indexes of the columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) # fit() and transform() at the same time 

#  remainder='passthrough' --> columns, other than the ones specified, will remain untouched

In [24]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [25]:
# Replace "Yes" or "No" by binary variables

# Label Encoder is only for labels/output
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y) # send one single matrix/vector, y doesn't necessarily need to be a numpy array

In [26]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [27]:
from sklearn.model_selection import train_test_split

# test size = 0.2 --> Train = 80% of data, test = 20% of data
# random_state = 0 --> to keep the random split the same - to make results reproducable -- like np.random.seed()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Train set --> train model on existing observations
# Test set --> evaluate model on new observations (future data)

# X_train corresponds to y_train
# X_test corresponds to y_test

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8, 5), (8,), (2, 5), (2,))

In [28]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [29]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [30]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [31]:
print(y_test)

[0 1]


## Feature Scaling

In [33]:
# Scale features to make sure all values are in the same scale (Prevent one feature from dominating the other)

# Feature scaling is done after train,test split (to prevent information leakage)

# Scaling on both train and test set must be same (Better to do at once). Although they are done seperately
# Scaling is not done on y(labels)

# Feature scaling is not needed for all ML algorithms. Only for some.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:]) # Take all rows, and columns from 3rd to last (excluding dummy variable columns) 
X_test[:, 3:] = sc.transform(X_test[:, 3:]) # only apply transform() because test state must be applied the same scale as train state
# in order to get the same transformation. (fit_transform would have given a new scale)

# IMP: Standardisation is not done on dummy variables (variables that have been one hot encoded)
# because we lose our numerical values

# Only apply them on numerical variables 


In [34]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [35]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
